<a href="https://colab.research.google.com/github/seirah-yang/BootCamp/blob/main/ML_Fullprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#1. 데이터 로드
import pandas as pd
from sklearn.linear_model import LinearRegression
import os

df = pd.read_csv("titanic.csv")
df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [5]:
#2. 결측치 확인#
dict(df.isnull().sum())

{'PassengerId': np.int64(0),
 'Survived': np.int64(0),
 'Pclass': np.int64(0),
 'Name': np.int64(0),
 'Sex': np.int64(0),
 'Age': np.int64(177),
 'SibSp': np.int64(0),
 'Parch': np.int64(0),
 'Ticket': np.int64(0),
 'Fare': np.int64(0),
 'Cabin': np.int64(687),
 'Embarked': np.int64(2)}

In [6]:
df["Age"].fillna(df["Age"].mean(), inplace=True)
df["Embarked"].fillna("S", inplace=True)

/tmp/ipython-input-1463978998.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].mean(), inplace=True)
/tmp/ipython-input-1463978998.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [7]:
dict(df.isnull().sum())

{'PassengerId': np.int64(0),
 'Survived': np.int64(0),
 'Pclass': np.int64(0),
 'Name': np.int64(0),
 'Sex': np.int64(0),
 'Age': np.int64(0),
 'SibSp': np.int64(0),
 'Parch': np.int64(0),
 'Ticket': np.int64(0),
 'Fare': np.int64(0),
 'Cabin': np.int64(687),
 'Embarked': np.int64(0)}

In [8]:
#3. 컬럼 셀렉션
df.columns
new_df = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp',  'Parch', 'Fare', 'Embarked']]

In [9]:
#성별 숫자화
new_df["Sex"] = new_df["Sex"].map(lambda x : 0 if 'male' else 1)

/tmp/ipython-input-2450466666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Sex"] = new_df["Sex"].map(lambda x : 0 if 'male' else 1)


In [10]:
#Embarked 숫자화
sdict = {'S':0, 'C':1, 'Q':2}
new_df['Embarked'] = new_df['Embarked'].map(lambda x: sdict[x])

/tmp/ipython-input-2894719560.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Embarked'] = new_df['Embarked'].map(lambda x: sdict[x])


In [11]:
new_df.head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.25,0


In [13]:
#4. K-means 파생변수
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, init = "k-means++") # kmeans++은 처음 센트로이드를 계산적으로 초기화함
new_df["cluster"] = km.fit_predict(new_df.drop("Survived", axis=1)) #survived colum을 제외한 나머지 컬럼만
            # 데이터를 줘서 3개로 군집 라벨링

/tmp/ipython-input-4248965331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["cluster"] = km.fit_predict(new_df.drop("Survived", axis=1)) #survived colum을 제외한 나머지 컬럼만


In [17]:
#5. scaling
#6. training-test-split
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(new_df.drop("Survived", axis=1),
                                                    new_df["Survived"],
                                                    test_size=0.2,
                                                    shuffle=True)
X_train = RobustScaler().fit_transform(X_train)
X_valid = RobustScaler().fit_transform(X_valid)

In [18]:
#AutoML
#pycaret으로 ahepf randomForest / DecisionTree/Logistic regression

In [19]:
!pip install optuna


In [28]:
#8. optuna
import optuna
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def DecisionTreeobjective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split = min_samples_split, random_state=42)
    model.fit(X_train, y_train)
    y_pred =  model.predict(X_valid)
    return f1_score(y_valid, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(DecisionTreeobjective, n_trials=20)
print("Best parameters:", study.best_params)

[I 2025-08-29 12:20:47,161] A new study created in memory with name: no-name-1304817e-c91b-42a0-ae76-d8cde69c0fa3
[I 2025-08-29 12:20:47,172] Trial 0 finished with value: 0.5571428571428572 and parameters: {'max_depth': 13, 'min_samples_split': 12}. Best is trial 0 with value: 0.5571428571428572.
[I 2025-08-29 12:20:47,181] Trial 1 finished with value: 0.5899280575539568 and parameters: {'max_depth': 17, 'min_samples_split': 16}. Best is trial 1 with value: 0.5899280575539568.
[I 2025-08-29 12:20:47,189] Trial 2 finished with value: 0.5857142857142857 and parameters: {'max_depth': 14, 'min_samples_split': 14}. Best is trial 1 with value: 0.5899280575539568.
[I 2025-08-29 12:20:47,198] Trial 3 finished with value: 0.5957446808510638 and parameters: {'max_depth': 16, 'min_samples_split': 16}. Best is trial 3 with value: 0.5957446808510638.
[I 2025-08-29 12:20:47,208] Trial 4 finished with value: 0.5413533834586466 and parameters: {'max_depth': 19, 'min_samples_split': 7}. Best is trial 3

Best parameters: {'max_depth': 16, 'min_samples_split': 16}


In [35]:

def RandomForestobjective(trial):
  n_estimators = trial.suggest_int("n_estimators", 50, 300)
  max_depth = trial.suggest_int("max_depth", 2, 20)
  model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_valid)
  return f1_score(y_valid, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(RandomForestobjective, n_trials=20)
print("Best parameters:", study.best_params)

[I 2025-08-29 12:25:45,617] A new study created in memory with name: no-name-c8c8137c-eb9d-43d5-bf9a-11e8fc621a1c
[I 2025-08-29 12:25:46,212] Trial 0 finished with value: 0.5555555555555556 and parameters: {'n_estimators': 262, 'max_depth': 17}. Best is trial 0 with value: 0.5555555555555556.
[I 2025-08-29 12:25:46,686] Trial 1 finished with value: 0.546875 and parameters: {'n_estimators': 213, 'max_depth': 15}. Best is trial 0 with value: 0.5555555555555556.
[I 2025-08-29 12:25:47,130] Trial 2 finished with value: 0.5225225225225225 and parameters: {'n_estimators': 253, 'max_depth': 4}. Best is trial 0 with value: 0.5555555555555556.
[I 2025-08-29 12:25:47,760] Trial 3 finished with value: 0.56 and parameters: {'n_estimators': 287, 'max_depth': 12}. Best is trial 3 with value: 0.56.
[I 2025-08-29 12:25:47,935] Trial 4 finished with value: 0.5225225225225225 and parameters: {'n_estimators': 94, 'max_depth': 5}. Best is trial 3 with value: 0.56.
[I 2025-08-29 12:25:48,139] Trial 5 finis

Best parameters: {'n_estimators': 133, 'max_depth': 12}


In [36]:
def Logisticobjective(trial):
  C = trial.suggest_loguniform("C", 1e-3, 10)
  model = LogisticRegression(C=C, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_valid)
  return f1_score(y_valid, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(Logisticobjective, n_trials=20)
print("Best parameters:", study.best_params)


[I 2025-08-29 12:25:58,844] A new study created in memory with name: no-name-971bcfca-4455-496a-b319-a612f9655aea
/tmp/ipython-input-1259242089.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 10)
[I 2025-08-29 12:25:58,866] Trial 0 finished with value: 0.5573770491803278 and parameters: {'C': 0.47261082126274523}. Best is trial 0 with value: 0.5573770491803278.
/tmp/ipython-input-1259242089.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 10)
[I 2025-08-29 12:25:58,880] Trial 1 finished with value: 0.5645161290322581 and parameters: {'C': 3.9272522462491173}. Best is trial 1 with valu

Best parameters: {'C': 3.9272522462491173}
